# **Cafe Sales Data Cleaning and Analysis Project**

This project involves cleaning a real-world dirty dataset from kaggle, a cafe's sales records, using Pandas and NumPy. After cleaning, I’ll perform exploratory data analysis (EDA) to extract insights.

## 1. Loading the Data

In this step, I load the dataset and perform a basic inspection to understand its structure.

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df=pd.read_csv("dirty_cafe_sales.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [6]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [7]:
df.shape

(10000, 8)

## 2. Data Types Analysis

From the data types of the attributes, it can be concluded that all columns are currently of type `object`. However, some columns should be converted to more appropriate data types:

- `Quantity`, `Price Per Unit`, and `Total Spent` should be converted to **numerical types** (e.g., `int` or `float`) to allow for mathematical operations and analysis.  
- `Transaction Date` should be converted to a **datetime format** to enable time-based analysis such as sales trends by day, week, or month.


## 3, Data Cleaning
This step involves handling missing values, handling duplicated values, converting data to proper types, and ensuring the data is ready for analysis.

In [8]:
df.isna().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

### Missing Values Analysis

From the output above, we can conclude that almost all columns contain missing values, except for `Transaction ID`. Each of these missing values will need to be handled appropriately based on the context and importance of the column.


### Handling Missing Values and Invalid Values in `Item` column

* For the missing values in the `Item` column, it would be appropriate to replace them with `"Unknown Item"`.
* For the values that have `"ERROR"` or `"UNKNOWN"`, it would also be appropriate to replace them with `"Unknown Item"`. 


In [9]:
df.loc[:,"Item"].unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

In [10]:
df["Item"]=df["Item"].fillna("Unknown Item")

In [11]:
df["Item"].isna().sum()

0

In [12]:
df["Item"]=np.where(df["Item"].isin(["ERROR","UNKNOWN"]),"Unknown Item",df["Item"])

### Handling Missing Values and Invalid Values in `Quantity`

- Missing values in the `Quantity` column will initially be replaced with `0` to handle invalid entries and allow for proper data type conversion.
- Entries like `"ERROR"` and `"UNKNOWN"` will be replaced with `0` as a placeholder.
- After converting the columns to the correct numeric format, these placeholder values can be further refined or updated based on related attributes if necessary.


In [13]:
df["Quantity"].unique()

array(['2', '4', '5', '3', '1', 'ERROR', 'UNKNOWN', nan], dtype=object)

In [14]:
df["Quantity"]=df["Quantity"].fillna(0)

In [15]:
df["Quantity"].isna().sum()

0

In [16]:
df["Quantity"]=np.where(df["Quantity"].isin(["ERROR","UNKNOWN"]),0,df["Quantity"])

In [17]:
df["Quantity"]=df["Quantity"].astype("int8")

### Handling Missing Values in `Price Per Unit`

- Missing values in the `Price Per Unit` column will initially be replaced with `0` to handle invalid entries and allow for proper data type conversion.
- Entries like `"ERROR"` and `"UNKNOWN"` will be replaced with `0` as a placeholder.
- After converting the columns to the correct numeric format, these placeholder values can be further refined or updated based on related attributes if necessary.

In [18]:
df["Price Per Unit"].unique()

array(['2.0', '3.0', '1.0', '5.0', '4.0', '1.5', nan, 'ERROR', 'UNKNOWN'],
      dtype=object)

In [19]:
df["Price Per Unit"]=np.where(df["Price Per Unit"].isin(["ERROR","UNKNOWN"]),0,df["Price Per Unit"])

In [20]:
df["Price Per Unit"]=df["Price Per Unit"].astype("float64") # Data type is converted to float to make it easier to work when filling the na values

#### Group-Based Imputation for `Price Per Unit` to fill the missing values

The DataFrame will be grouped by the `Item` column to calculate the average `Price Per Unit` for each item. These group-specific averages will then be used to fill in missing values in the `Price Per Unit` column. This method ensures the imputed values are contextually accurate while preserving the original number of rows in the dataset.


In [21]:
df["Price Per Unit"]=df.groupby('Item')["Price Per Unit"].transform(lambda x: x.fillna(x.mean()))#mean of the price for each item

### Handling Missing Values and Invalid Values in `Total Spent`

* Missing values in the `Total Spent` column will initially be replaced with `0` to handle invalid entries and allow for proper data type conversion.
* Entries like `"ERROR"` and `"UNKNOWN"` will be replaced with `0` as a placeholder.
* After converting the columns to the correct numeric format, these placeholder values can be further refined or updated based on related attributes if necessary.

In [22]:
df["Total Spent"].unique()

array(['4.0', '12.0', 'ERROR', '10.0', '20.0', '9.0', '16.0', '15.0',
       '25.0', '8.0', '5.0', '3.0', '6.0', nan, 'UNKNOWN', '2.0', '1.0',
       '7.5', '4.5', '1.5'], dtype=object)

In [23]:
df["Total Spent"]=df["Total Spent"].fillna(0)

In [24]:
df["Total Spent"]=np.where(df["Total Spent"].isin(["ERROR","UNKNOWN"]),0,df["Total Spent"])

In [25]:
df["Total Spent"]=df["Total Spent"].astype("float64")

In [26]:
df.isna().sum()

Transaction ID         0
Item                   0
Quantity               0
Price Per Unit         0
Total Spent            0
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

### Handling Missing and Invalid Values in `Payment Method`

- Missing values in the `Payment Method` column are replaced with `"Unknown"` since the transaction was completed, indicating the customer must have paid in some way (it's just not recorded).
- Entries labeled as `"ERROR"` are kept as they are to preserve data integrity.


In [27]:
df["Payment Method"].unique()

array(['Credit Card', 'Cash', 'UNKNOWN', 'Digital Wallet', 'ERROR', nan],
      dtype=object)

In [28]:
df["Payment Method"]=df["Payment Method"].fillna("UNKNOWN")

### Handling Missing and Invalid Values in `Location`

- Missing values in the `Location` column are replaced with `"Unknown"` since the transaction still occurred, even if the specific location wasn't recorded.
- Entries labeled as `"ERROR"` are left as-is to indicate potential data issues that might need manual review or further clarification.


In [29]:
df["Location"].unique()

array(['Takeaway', 'In-store', 'UNKNOWN', nan, 'ERROR'], dtype=object)

In [30]:
df["Location"]=df["Location"].fillna("UNKNOWN")

### Handling Missing and Invalid Values in `Transaction Date`

In [31]:
df["Transaction Date"].unique()

array(['2023-09-08', '2023-05-16', '2023-07-19', '2023-04-27',
       '2023-06-11', '2023-03-31', '2023-10-06', '2023-10-28',
       '2023-07-28', '2023-12-31', '2023-11-07', 'ERROR', '2023-05-03',
       '2023-06-01', '2023-03-21', '2023-11-15', '2023-06-10',
       '2023-02-24', '2023-03-25', '2023-01-15', '2023-04-04',
       '2023-03-30', '2023-12-01', '2023-09-18', '2023-06-03',
       '2023-12-13', '2023-04-20', '2023-04-10', '2023-03-11',
       '2023-06-02', '2023-11-06', '2023-08-15', '2023-10-09',
       '2023-05-28', '2023-07-17', '2023-04-29', '2023-06-08',
       '2023-06-29', '2023-04-17', '2023-12-22', '2023-01-10',
       '2023-10-02', '2023-02-23', '2023-03-22', '2023-11-03',
       '2023-03-02', '2023-06-26', '2023-05-02', '2023-09-05',
       '2023-01-08', '2023-03-15', '2023-11-25', '2023-12-05',
       '2023-03-19', '2023-06-27', '2023-04-19', '2023-10-07',
       '2023-09-30', '2023-05-27', '2023-11-18', '2023-10-20',
       '2023-10-03', '2023-10-27', '2023-04-06

* First, we converted all values in the column to lowercase strings and removed any leading or trailing spaces:

In [32]:
df["Transaction Date"]=df["Transaction Date"].astype("str").str.lower().str.strip()

* Then, we used `pd.to_datetime()` with `errors='coerce'` to convert valid date strings to datetime format and set any invalid entries to `NaT` (Not a Time), which behaves like `NaN` for datetime objects.

In [33]:
df["Transaction Date"]=pd.to_datetime(df["Transaction Date"],errors="coerce") 
#This will convert any values that are not date to NaT which is like NaN but for time

In [34]:
df["Transaction Date"].isna().sum()

460

* Finally, we filled these missing dates with a placeholder date of `1900-01-01`.

In [35]:
df["Transaction Date"]=df["Transaction Date"].fillna(pd.Timestamp("1900-01-01"))# A place holder for the unknown dates

In [36]:
df.isna().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

### Since now there are no missing values it is time to calculate the correct numerical values for 
* `Quantity`
* `Price Per Unit`
* `Total Spent`

### 1.Calculating the correct **`Total Spent`**

#### The correct `Total Spent` will be calculated in cases where `0` was initially used as a placeholder. This is done by multiplying `Price Per Unit` and `Quantity`, only when both values are not equal to `0`.


In [37]:
df["Total Spent"]=np.where(
    (df["Total Spent"]==0) & 
    (df["Price Per Unit"]!=0) & 
    (df["Quantity"]!=0),
    df["Price Per Unit"]*df["Quantity"],
    df["Total Spent"])

### 2.Calculating the correct **`Price Per Unit`**

#### The correct `Price Per Unit` will be calculated in cases where `0` was initially used as a placeholder. This is done by dividing `Total Spent` and `Quantity`, only when both values are not equal to `0`.

In [38]:
df["Price Per Unit"]=np.where(
    (df["Price Per Unit"]==0) & 
    (df["Total Spent"]!=0) & 
    (df["Quantity"]!=0),
    df["Total Spent"]/df["Quantity"],
    df["Price Per Unit"])

### 3.Calculating the correct **`Quantity`**

#### The correct `Quantity` will be calculated in cases where `0` was initially used as a placeholder. This is done by dividing `Total Spent` and `Price Per Unit`, only when both values are not equal to `0`.

In [39]:
df["Quantity"]=np.where(
    (df["Quantity"]==0) & 
    (df["Total Spent"]!=0) & 
    (df["Price Per Unit"]!=0),
    df["Total Spent"]/df["Price Per Unit"],
    df["Quantity"])

In [40]:
df["Quantity"]=df["Quantity"].astype("int8") #It needxs to be countable number as it is quantity

## Now it is time to fill the remaining `0` values that are used as place holders if it is possible

In [41]:
df.Item.unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'Unknown Item',
       'Sandwich', 'Juice', 'Tea'], dtype=object)

#### It is possible to fill the `Price Per Unit` for `Item` coffee with `2` because it is frequent.

In [42]:
df[df.Item=="Coffee"]["Price Per Unit"].value_counts()

Price Per Unit
2.000000    1144
1.933682      19
0.000000       2
Name: count, dtype: int64

In [43]:
df['Price Per Unit']=np.where(df["Item"]=="Coffee",2,df["Price Per Unit"]) #For the coffee we are filling 2 as the Price Per Unit

#### It is possible to fill the `Price Per Unit` for `Item` cake with `3` because it is frequent.

In [44]:
df[df.Item=="Cake"]["Price Per Unit"].value_counts()

Price Per Unit
3.000000    1116
2.911449      21
0.000000       2
Name: count, dtype: int64

In [45]:
df['Price Per Unit']=np.where(df["Item"]=="Cake",3,df["Price Per Unit"])

#### It is possible to fill the `Price Per Unit` for `Item` Cookie with `1` because it is frequent.

In [46]:
df[df.Item=="Cookie"]["Price Per Unit"].value_counts()

Price Per Unit
1.000000    1063
0.960674      24
0.000000       5
Name: count, dtype: int64

In [47]:
df['Price Per Unit']=np.where(df["Item"]=="Cookie",1,df["Price Per Unit"])

#### It is possible to fill the `Price Per Unit` for `Item` Salad with `5` because it is frequent.

In [48]:
df[df.Item=="Salad"]["Price Per Unit"].value_counts()

Price Per Unit
5.000000    1131
4.779152      16
0.000000       1
Name: count, dtype: int64

In [49]:
df['Price Per Unit']=np.where(df["Item"]=="Salad",5,df["Price Per Unit"])

#### It is possible to fill the `Price Per Unit` for `Item` Smoothie with `4` because it is frequent.

In [50]:
df[df.Item=="Smoothie"]["Price Per Unit"].value_counts()

Price Per Unit
4.000000    1068
3.865672      24
0.000000       4
Name: count, dtype: int64

In [51]:
df['Price Per Unit']=np.where(df["Item"]=="Smoothie",4,df["Price Per Unit"])

#### It is possible to fill the `Price Per Unit` for `Item` Sandwich with `4` because it is frequent.

In [52]:
df[df.Item=="Sandwich"]["Price Per Unit"].value_counts()

Price Per Unit
4.000000    1111
3.885099      17
0.000000       3
Name: count, dtype: int64

In [53]:
df['Price Per Unit']=np.where(df["Item"]=="Sandwich",4,df["Price Per Unit"])

#### It is possible to fill the `Price Per Unit` for `Item` Juice with `3` because it is frequent.

In [54]:
df[df.Item=="Juice"]["Price Per Unit"].value_counts()

Price Per Unit
3.000000    1154
2.885615      17
Name: count, dtype: int64

In [55]:
df['Price Per Unit']=np.where(df["Item"]=="Juice",3,df["Price Per Unit"])

#### It is possible to fill the `Price Per Unit` for `Item` Tea with `1.5` because it is frequent.

In [56]:
df[df.Item=="Tea"]["Price Per Unit"].value_counts()

Price Per Unit
1.500000    1064
1.435454      20
0.000000       5
Name: count, dtype: int64

In [57]:
df['Price Per Unit']=np.where(df["Item"]=="Tea",1.5,df["Price Per Unit"])

### Now the data have known prices except where the items are unknown

In [58]:
df[df['Price Per Unit']==0]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
1761,TXN_3611851,Unknown Item,4,0.0,0.0,Credit Card,UNKNOWN,2023-02-09
2289,TXN_7524977,Unknown Item,4,0.0,0.0,ERROR,UNKNOWN,2023-12-09
3779,TXN_7376255,Unknown Item,0,0.0,25.0,UNKNOWN,In-store,2023-05-27
4152,TXN_9646000,Unknown Item,2,0.0,0.0,UNKNOWN,In-store,2023-12-14


### The `Total Spent` can be recalculated now with the new `Price Per Unit`

In [59]:
df[df['Total Spent']==0]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
236,TXN_8562645,Salad,0,5.0,0.0,UNKNOWN,In-store,2023-05-18
278,TXN_3229409,Juice,0,3.0,0.0,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,0,3.0,0.0,UNKNOWN,UNKNOWN,2023-03-17
738,TXN_8696094,Sandwich,0,4.0,0.0,UNKNOWN,Takeaway,2023-05-14
1761,TXN_3611851,Unknown Item,4,0.0,0.0,Credit Card,UNKNOWN,2023-02-09
2229,TXN_8498613,Sandwich,2,4.0,0.0,UNKNOWN,UNKNOWN,2023-11-08
2289,TXN_7524977,Unknown Item,4,0.0,0.0,ERROR,UNKNOWN,2023-12-09
2585,TXN_1259340,Tea,3,1.5,0.0,Digital Wallet,UNKNOWN,2023-02-24
2796,TXN_9188692,Cake,0,3.0,0.0,Credit Card,UNKNOWN,2023-12-01
3203,TXN_4565754,Smoothie,0,4.0,0.0,Digital Wallet,Takeaway,2023-10-06


In [60]:
df['Total Spent']=np.where(
    (df["Total Spent"]==0) & 
    (df["Price Per Unit"]!=0) & 
    (df["Quantity"]!=0),
    df["Price Per Unit"]*df["Quantity"],
    df["Total Spent"])


### The `Quantity` can be recalculated now with the new `Price Per Unit` and `Total Spent`

In [61]:
df[df['Quantity']==0]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
236,TXN_8562645,Salad,0,5.0,0.0,UNKNOWN,In-store,2023-05-18
278,TXN_3229409,Juice,0,3.0,0.0,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,0,3.0,0.0,UNKNOWN,UNKNOWN,2023-03-17
738,TXN_8696094,Sandwich,0,4.0,0.0,UNKNOWN,Takeaway,2023-05-14
1436,TXN_7590801,Tea,0,1.5,6.0,Cash,Takeaway,1900-01-01
2330,TXN_3849488,Salad,0,5.0,5.0,UNKNOWN,In-store,2023-03-01
2796,TXN_9188692,Cake,0,3.0,0.0,Credit Card,UNKNOWN,2023-12-01
3203,TXN_4565754,Smoothie,0,4.0,0.0,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,0,2.0,0.0,UNKNOWN,UNKNOWN,2023-04-07
3401,TXN_3251829,Tea,0,1.5,0.0,Digital Wallet,In-store,2023-07-25


In [62]:
df['Quantity']=np.where(
    (df["Quantity"]==0) & 
    (df["Price Per Unit"]!=0) & 
    (df["Total Spent"]!=0),
    df["Total Spent"]/df["Price Per Unit"],
    df["Quantity"])

In [ ]:
df["Quantity"]=df["Quantity"].astype("int8")

In [63]:
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2.0,2.000000,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.000000,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.000000,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.000000,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.000000,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2.0,2.000000,4.0,UNKNOWN,UNKNOWN,2023-08-30
9996,TXN_9659401,Unknown Item,3.0,2.831224,3.0,Digital Wallet,UNKNOWN,2023-06-02
9997,TXN_5255387,Coffee,4.0,2.000000,8.0,Digital Wallet,UNKNOWN,2023-03-02
9998,TXN_7695629,Cookie,3.0,1.000000,3.0,Digital Wallet,UNKNOWN,2023-12-02


In [64]:
df[df['Quantity']==0]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
236,TXN_8562645,Salad,0.0,5.0,0.0,UNKNOWN,In-store,2023-05-18
278,TXN_3229409,Juice,0.0,3.0,0.0,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,0.0,3.0,0.0,UNKNOWN,UNKNOWN,2023-03-17
738,TXN_8696094,Sandwich,0.0,4.0,0.0,UNKNOWN,Takeaway,2023-05-14
2796,TXN_9188692,Cake,0.0,3.0,0.0,Credit Card,UNKNOWN,2023-12-01
3203,TXN_4565754,Smoothie,0.0,4.0,0.0,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,0.0,2.0,0.0,UNKNOWN,UNKNOWN,2023-04-07
3401,TXN_3251829,Tea,0.0,1.5,0.0,Digital Wallet,In-store,2023-07-25
3779,TXN_7376255,Unknown Item,0.0,0.0,25.0,UNKNOWN,In-store,2023-05-27
4257,TXN_6470865,Coffee,0.0,2.0,0.0,Digital Wallet,Takeaway,2023-09-18


In [65]:
df[df['Price Per Unit']==0]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
1761,TXN_3611851,Unknown Item,4.0,0.0,0.0,Credit Card,UNKNOWN,2023-02-09
2289,TXN_7524977,Unknown Item,4.0,0.0,0.0,ERROR,UNKNOWN,2023-12-09
3779,TXN_7376255,Unknown Item,0.0,0.0,25.0,UNKNOWN,In-store,2023-05-27
4152,TXN_9646000,Unknown Item,2.0,0.0,0.0,UNKNOWN,In-store,2023-12-14


In [137]:
df[df['Total Spent']==0]

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
236,TXN_8562645,Salad,0,5.0,0.0,UNKNOWN,In-store,2023-05-18
278,TXN_3229409,Juice,0,3.0,0.0,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,0,3.0,0.0,UNKNOWN,UNKNOWN,2023-03-17
738,TXN_8696094,Sandwich,0,4.0,0.0,UNKNOWN,Takeaway,2023-05-14
1761,TXN_3611851,Unknown Item,4,0.0,0.0,Credit Card,UNKNOWN,2023-02-09
2289,TXN_7524977,Unknown Item,4,0.0,0.0,ERROR,UNKNOWN,2023-12-09
2796,TXN_9188692,Cake,0,3.0,0.0,Credit Card,UNKNOWN,2023-12-01
3203,TXN_4565754,Smoothie,0,4.0,0.0,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,0,2.0,0.0,UNKNOWN,UNKNOWN,2023-04-07
3401,TXN_3251829,Tea,0,1.5,0.0,Digital Wallet,In-store,2023-07-25


### Final Note on Data Cleaning

*Now the data is cleaned. A value of `0` in any of the numerical columns indicates that the original entry was either `NaN`, `ERROR`, or `UNKNOWN` before the cleaning process.*


### Analytical Questions

#### 1. Which items bring in the highest total revenue?

In [87]:
item_with_highest_revenue=df.groupby(["Item"])[["Total Spent"]].sum().sort_values(by="Total Spent", ascending=False).round(1).iloc[0,:]

In [88]:
item_with_highest_revenue

Total Spent    17343.9
Name: Salad, dtype: float64

### Item with the Highest Revenue
#### The item that brings in the most revenue is **`Salad`**, with a total revenue of $`17,343.90`.


### 2. Which items are sold most frequently (by quantity)?

In [96]:
item_frequently_sold=df.groupby("Item")[["Quantity"]].sum().sort_values(by="Quantity", ascending=False).iloc[0,:]

In [98]:
item_frequently_sold

Quantity    3536
Name: Coffee, dtype: int64

### Item Frequently Sold

#### The item that is frequently sold is **`Coffee`**, with a total quantity of `3536`.

### 3. What is the average total spent per transaction?

In [106]:
avg_amount_spent_per_transaction = df["Total Spent"].mean().round(1)

In [107]:
avg_amount_spent_per_transaction

8.9

In [111]:
df["Total Spent"].median()

8.0

In [114]:
df["Total Spent"].std()

6.0129697706803045

#### Average Total Spent Per Transaction

The average total spent per transaction is **`$8.90`**, with a standard deviation of **`$6.01`**.  
This indicates that spending varies considerably between transactions. While some customers make small purchases, others spend significantly more, suggesting a wide range of spending behavior **`($8.90 ± $6.01)`**.


### 4.Which location generates the most revenue overall?

In [118]:
location_with_highest_revenue=df.groupby("Location")[["Total Spent"]].sum().sort_values(by="Total Spent", ascending=False)

In [119]:
location_with_highest_revenue

,Total Spent
Location,
UNKNOWN,32128.000000
In-store,27172.421963
Takeaway,26552.005578
ERROR,3241.500000


### `Location` with the Highest Revenue
* `UNKNOWN` is the top revenue-generating location.
However, this category likely includes transactions where the original location data was missing or labeled as `"UNKNOWN"`, and therefore it lacks clarity. It may represent online orders, third-party apps, or any unspecified channel.

* `ERROR` has the lowest revenue, which may indicate:
Transactions that were partially completed,
 system issues during logging
 or cases where payment still went through despite an error.

**NOTICE**
* When interpreting the `UNKNOWN` and `ERROR` categories, it's important to acknowledge the data limitations and avoid over-reliance on incomplete labels.

### 5. Which days of the week have the highest number of transactions?

In [144]:
day_with_high_revenue=df.groupby(df[df["Transaction Date"]!="1900-01-01"]["Transaction Date"].dt.dayofweek)[["Total Spent"]].sum().sort_values(by="Total Spent", ascending=False)

In [145]:
day_with_high_revenue

,Total Spent
Transaction Date,
3.0,12401.271231
4.0,12333.655296
6.0,12287.500000
0.0,12139.870907
5.0,12039.500000
1.0,12039.234347
2.0,11679.395760


In [147]:
day_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

In [148]:
day_with_high_revenue.index=day_with_high_revenue.index.map(day_map)

In [149]:
day_with_high_revenue

,Total Spent
Transaction Date,
Thursday,12401.271231
Friday,12333.655296
Sunday,12287.500000
Monday,12139.870907
Saturday,12039.500000
Tuesday,12039.234347
Wednesday,11679.395760


### Day-wise Revenue Analysis

* The day with the **highest total revenue** is **Thursday**.
* The day with the **lowest total revenue** is **Wednesday**.




### 6. Which payment method is used most often?

In [151]:
freq_paym_meth=df["Payment Method"].value_counts()

In [153]:
freq_paym_meth

Payment Method
UNKNOWN           2872
Digital Wallet    2291
Credit Card       2273
Cash              2258
ERROR              306
Name: count, dtype: int64

### Most Frequently Used Payment Method

- `UNKNOWN` is the most frequently recorded payment method. However, this label likely includes entries where the actual method was missing, mislabeled, or not properly captured.

- `Digital Wallet`, `Credit Card`, and `Cash` follow as the most common explicit payment methods, indicating a diverse range of customer preferences.

- `ERROR` entries are minimal and may reflect system glitches or improperly logged transactions.

**NOTICE**

Just like with location data, the `UNKNOWN` and `ERROR` categories should be interpreted cautiously, as they represent data quality issues rather than true customer behavior.


### 7. Which items are most often purchased in bulk (highest average quantity)?

In [142]:
bulk_items = df[df["Quantity"] > 0].groupby("Item")[["Quantity"]].mean().sort_values(by='Quantity',ascending=False)


In [143]:
bulk_items

,Quantity
Item,
Smoothie,3.049360
Coffee,3.048276
Cake,3.047452
Sandwich,3.042591
Tea,3.039522
Salad,3.027051
Unknown Item,3.004132
Juice,3.000856
Cookie,2.965138


**Note**
All listed items have an average `Quantity` of around `3`, showing a consistent trend in bulk ordering behavior across product types. This may suggest frequent group or repeat purchases.

### 8. How has revenue trended over months? 

In [161]:
highest_monthly_revenue=df.groupby(df[df["Transaction Date"]!="1900-01-01"]["Transaction Date"].dt.month)[["Total Spent"]].sum().sort_values(by="Total Spent", ascending=False)

In [162]:
highest_monthly_revenue

,Total Spent
Transaction Date,
6.0,7352.870907
10.0,7313.655296
1.0,7254.000000
3.0,7216.000000
4.0,7178.734347
12.0,7177.000000
8.0,7111.395760
11.0,6967.000000
5.0,6957.500000


In [157]:
month_map = {1: "January",2: "February",3: "March",4: "April",5: "May",6: "June",7: "July",8: "August",9: "September",
    10: "October",11: "November",12: "December"
}

In [163]:
highest_monthly_revenue.index=highest_monthly_revenue.index.map(month_map)

In [164]:
highest_monthly_revenue

,Total Spent
Transaction Date,
June,7352.870907
October,7313.655296
January,7254.000000
March,7216.000000
April,7178.734347
December,7177.000000
August,7111.395760
November,6967.000000
May,6957.500000


### Month-wise Revenue Analysis

* The month with the **highest total revenue** is **June**.
* The month with the **lowest total revenue** is **February**.


##  Final Report: Cafe Sales Data Analysis

### 1. Item with the Highest Revenue
The item that generated the highest total revenue is `Salad`, with a total revenue of `$17,343.90`.

### 2. Most Frequently Sold Item
The most frequently sold item is `Coffee`, making it the most popular product by quantity.

### 3. Average Total Spent per Transaction
- The average amount spent per transaction is `$8.90`.

- However, due to a high standard deviation, individual transaction totals vary significantly, generally ranging between `$2 and $14`.

### 4. Location Generating the Most Revenue
- The `Location` with the highest revenue is labeled `"UNKNOWN"`.

- This category likely includes entries where the original location data was missing or unspecified.

- It may represent online orders, third-party delivery apps, or other unclassified channels.

- The "ERROR" location generated the lowest revenue.

- This may reflect system logging issues, incomplete transactions, or exceptions during payment.

### 5. Day of the Week with Most Transactions
- The day with the highest number of transactions is `Thursday`.

- The lowest total spent occurs on Wednesday.

### 6. Most Used Payment Method
- The most commonly used payment method is `"UNKNOWN"`, due to a large number of initially missing values replaced as such.

- Excluding `"UNKNOWN"`, the most frequently used method is Digital Wallet.

### 7. Items Purchased Most in Bulk
- Items are typically purchased in similar quantities.

- After calculating the average quantity per item, all items had an average of approximately 3 units per transaction.

- This suggests that no item stands out significantly as being purchased in bulk more than others.

### 8. Revenue Trend Over Time `Monthly`
- The month with the highest revenue is June, followed by October.

- The lowest revenue was recorded in February.